In [2]:
# import libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [3]:
df_rate = pd.read_excel("標準普爾最新信用評級.xls", header=7)
dict_rate = dict(zip(df_rate['Exchange:Ticker'], df_rate['S&P Entity Credit Rating - Issuer Credit Rating - Local Currency LT [Latest] (Rating)']))
feature_names = [col.replace('[', '').replace(']', '').replace('<', '') for col in pd.read_excel(f"NEW財務數據/財務數據/2019財務數據.xls", header=7).columns]

In [4]:
def data_processing(year):
    
    df = pd.read_excel(f"NEW財務數據/財務數據/{year}財務數據.xls", header=7)
    
    # replace columns as 2019 columns
    df.columns = feature_names
    
    # map Exchange:Ticker to credit rating
    df["rate"] = df["Exchange:Ticker"].map(dict_rate)

    # Drop the following columns
    df = df.drop(columns=["Company Name", "Security Tickers","Exchange:Ticker"])    

    # Create a rating map dictionary
    rating_map = {
        'AAA': 1, 'AA+': 1, 'AA': 1, 'AA-': 1, 'A+': 1, 'A': 1, 'A-': 1,
        'BBB+': 2, 'BBB': 2, 'BBB-': 2,
        'BB+': 3, 'BB': 3, 'BB-': 3,
        'B+': 4, 'B': 4, 'B-': 4, 'CCC+': 4, 'CCC': 4, 'CCC-': 4, 'D': 4,
        'NR': np.nan
    }

    # Map the credit rating values to numerical values 
    df['rate'] = df['rate'].map(lambda x: rating_map.get(x, x))
    
    # drop the nan in rate column
    df = df.dropna(subset=['rate'])

    # Replace '-' with NaN values in all columns
    for col in df.columns:
        df[col] = df[col].replace('-', np.nan)

    # Replace 'NM' with NaN values
    df = df.replace('NM', np.nan)

    # Fill NaN values with the mean
    for col in df.columns:
        
        df[col] = df[col].fillna(df[col].mean())
    
    df = df.dropna(axis=1, how='all')

    return df

In [5]:
df = pd.DataFrame()

for year in [2019,2020,2021,2022]:
    # Concatenate the DataFrames
    df = pd.concat([df, data_processing(year)], ignore_index=True)

In [6]:
df

,"Total Assets - Capital IQ Latest Annual - 5 ($USDmm, Historical rate)","Total Capital - Capital IQ Latest Annual - 5 ($USDmm, Historical rate)","Sales/Sq. Ft., All (Net) - Capital IQ Latest Annual - 5 ($USD, Historical rate)","Total Equity - Capital IQ Latest Annual - 5 ($USDmm, Historical rate)","Total Enterprise Value My Setting Latest - 3 Year(s) ($USDmm, Historical rate)","Capital Expenditures - Compustat LTM - 3 ($USDmm, Historical rate)","Market Capitalization My Setting Latest - 3 Year(s) ($USDmm, Historical rate)",Shares Outstanding My Setting Latest - 3 Year(s) (mm),"Earnings from Cont. Ops., 1 Yr Growth % - Compustat LTM - 3 (%)","Net Income - Capital IQ LTM - 3 ($USDmm, Historical rate)",...,"Cash Dividends - Compustat LTM - 3 ($USDmm, Historical rate)",Effective Tax Rate - Capital IQ LTM - 3 (%),"Total Current Assets - Capital IQ Latest Annual - 5 ($USDmm, Historical rate)","Book Value/Share - Capital IQ Latest Annual - 5 ($USD, Historical rate)","Total Debt - Capital IQ Latest Annual - 5 ($USDmm, Historical rate)","Long-Term Debt - Capital IQ Latest Annual - 5 ($USDmm, Historical rate)","Common Dividends Paid - Capital IQ LTM - 3 ($USDmm, Historical rate)","Basic EPS - Capital IQ LTM - 3 ($USD, Historical rate)",Payout Ratio - Capital IQ LTM - 3 (%),rate
0,36500.0,24593.000000,315.180,9848.0,131552.900000,-1793.000,117563.9,579.7,48.107144,-1465.0,...,-3324.0,35.442712,13709.0,17.00,14745.0,13428.000000,-3324.000000,-2.630,62.669630,2.0
1,1524.7,1113.500000,315.180,936.3,1602.400000,-29.500,1434.8,34.9,14.400000,90.2,...,0.0,25.900000,942.7,27.50,177.2,177.200000,-813.538593,2.570,62.669630,3.0
2,67173.0,50288.000000,315.180,30722.0,205588.300000,-1964.000,195290.3,1776.8,48.107144,5161.0,...,-3429.0,17.800000,14632.0,17.40,19566.0,19359.000000,-3429.000000,2.950,66.400000,1.0
3,59352.0,31864.000000,315.180,-8446.0,279650.400000,-743.000,203857.4,1766.2,48.107144,8685.0,...,-10296.0,16.800000,16945.0,-5.71,40310.0,35002.000000,-10296.000000,4.880,118.500000,1.0
4,2385.6,1515.400000,315.180,1218.6,3172.500000,-194.800,2652.3,61.5,33.500000,109.6,...,0.0,45.100000,1336.0,18.30,296.8,296.800000,-813.538593,2.210,62.669630,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4179,93200.0,17659.077567,419.175,7463.0,43636.081179,-113.000,4478.6,148.1,51.930497,680.0,...,-282.0,23.300000,13919.0,46.30,2188.0,1008.000000,-829.574811,4.350,41.500000,2.0
4180,398.6,260.300000,419.175,235.0,1727.500000,-0.918,1687.1,100.9,51.930497,49.1,...,0.0,30.400000,310.6,1.99,25.3,7961.290928,-829.574811,0.487,89.866451,4.0
4181,13900.0,11333.000000,419.175,4544.0,81158.700000,-732.000,76502.7,462.1,10.800000,2344.0,...,-692.0,20.300000,6930.0,9.61,6789.0,6597.000000,-692.000000,5.080,29.500000,2.0
4182,6852.9,3306.000000,419.175,1997.9,11411.200000,-26.500,10738.2,402.3,69.800000,107.3,...,0.0,72.400000,545.7,4.95,1308.1,1235.700000,-829.574811,0.270,89.866451,3.0


In [6]:
# 四種分類百分比

rate_counts  = df['rate'].value_counts()
rate_percentages = (rate_counts / rate_counts.sum()) * 100
rate_percentages

2.0    35.181644
3.0    30.401530
4.0    21.606119
1.0    12.810707
Name: rate, dtype: float64

In [7]:
le = LabelEncoder()

X = df.drop('rate', axis=1)
y = le.fit_transform(df['rate'])

# 對 X 資料標準化
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Create the XGBoost model
model = xgb.XGBClassifier(
    objective='binary:logistic',
    max_depth=3,
    learning_rate=0.1,
    n_estimators=1000,
    random_state=42
)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the validation set
y_val_pred = model.predict(X_test)

# Calculate evaluation metrics
acc = accuracy_score(y_test, y_val_pred)                             # Calculate the accuracy score
f1 = f1_score(y_test, y_val_pred, average='weighted')                # Calculate the weighted F1-score
precision = precision_score(y_test, y_val_pred, average='weighted')  # Calculate the weighted precision
recall = recall_score(y_test, y_val_pred, average='weighted')        # Calculate the weighted recall
cm = confusion_matrix(y_test, y_val_pred)                            # Calculate the confusion matrix

# Print the evaluation metrics
print(f"Accuracy: {acc:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print("Confusion Matrix:")
print(cm)

Accuracy: 0.91
F1-score: 0.91
Precision: 0.91
Recall: 0.91
Confusion Matrix:
[[ 94  12   3   0]
 [  2 265  14   1]
 [  0  15 236   6]
 [  0   1  24 164]]
